In [248]:
import pandas as pd
import requests
import json
import time
import datetime
import sqlite3
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
import os

In [84]:
# Env data

fname = '/Users/xangma/environmentaldata.csv'
df_env = pd.read_csv(fname,parse_dates=True, low_memory=False)

df_env['sample.sampleDateTime'] = pd.to_datetime(df_env['sample.sampleDateTime'],format= '%Y-%m-%d %H:%M:%S')
df_env['Timestamp'] = df_env['sample.sampleDateTime'].map(pd.Timestamp.timestamp)
df_env['sampleDateTime'] = df_env['sample.sampleDateTime']

In [85]:
# Southern Water Data

fname = 'southern-water_spill-data_2017-20.csv'
df_sw = pd.read_csv(fname,parse_dates=True)

df_sw.loc[df_sw['Start Time'].str.len() < 8, 'Start Time'] = df_sw['Start Time'] + ':00'
df_sw.loc[df_sw['End Time'].str.len() < 8, 'End Time'] = df_sw['End Time'] + ':00'

df_sw['StartDateTime'] = df_sw['Start Date'] + ' ' +  df_sw['Start Time']
df_sw['EndDateTime'] = df_sw['End Date'] + ' ' +  df_sw['End Time']

df_sw['StartDateTime'] = pd.to_datetime(df_sw['StartDateTime'],format= '%d/%m/%Y %H:%M:%S')
df_sw['EndDateTime'] = pd.to_datetime(df_sw['EndDateTime'],format= '%d/%m/%Y %H:%M:%S')

In [86]:
df_env

,id,sample.samplingPoint,sample.samplingPoint.notation,sample.samplingPoint.label,sample.sampleDateTime,determinand.label,determinand.definition,determinand.notation,resultQualifier.notation,result,...,determinand.unit.label,sample.sampledMaterialType.label,sample.isComplianceSample,sample.purpose.label,sample.samplingPoint.easting,sample.samplingPoint.northing,yr,month,day,Timestamp
0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,SO-19801A08,APSLEY FARM BOREHOLE A,2000-03-22 10:30:00,BOD ATU,BOD : 5 Day ATU,85,<,3.00,...,mg/l,UNCODED,False,WASTE MONITORING (AGENCY AUDIT - PERMIT),442350.0,146570,2000,3,22,9.537210e+08
1,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,SO-19801A08,APSLEY FARM BOREHOLE A,2000-03-22 10:30:00,Ammonia(N),Ammoniacal Nitrogen as N,111,<,0.50,...,mg/l,UNCODED,False,WASTE MONITORING (AGENCY AUDIT - PERMIT),442350.0,146570,2000,3,22,9.537210e+08
2,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,SO-19801A08,APSLEY FARM BOREHOLE A,2000-03-22 10:30:00,Chloride Ion,Chloride,172,NaN,14.10,...,mg/l,UNCODED,False,WASTE MONITORING (AGENCY AUDIT - PERMIT),442350.0,146570,2000,3,22,9.537210e+08
3,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,SO-19801A08,APSLEY FARM BOREHOLE A,2000-03-22 10:30:00,pH,pH,61,NaN,7.15,...,phunits,UNCODED,False,WASTE MONITORING (AGENCY AUDIT - PERMIT),442350.0,146570,2000,3,22,9.537210e+08
4,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,SO-19801A08,APSLEY FARM BOREHOLE A,2000-03-22 10:30:00,COD as O2,Chemical Oxygen Demand :- {COD},92,NaN,27.00,...,mg/l,UNCODED,False,WASTE MONITORING (AGENCY AUDIT - PERMIT),442350.0,146570,2000,3,22,9.537210e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7764740,NaN,http://environment.data.gov.uk/water-quality/i...,SW-Z9420104,URCHFONT STW (HUMUS),2020-03-03 08:35:00,Sld Sus@105C,"Solids, Suspended at 105 C",135,<,5.00,...,mg/l,FINAL SEWAGE EFFLUENT,True,WATER QUALITY OPERATOR SELF MONITORING COMPLIA...,403930.0,157510,2020,3,3,1.583224e+09
7764741,NaN,http://environment.data.gov.uk/water-quality/i...,SW-Z9420104,URCHFONT STW (HUMUS),2020-03-03 08:35:00,pH,pH,61,NaN,7.96,...,phunits,FINAL SEWAGE EFFLUENT,True,WATER QUALITY OPERATOR SELF MONITORING COMPLIA...,403930.0,157510,2020,3,3,1.583224e+09
7764742,NaN,http://environment.data.gov.uk/water-quality/i...,SW-Z9420104,URCHFONT STW (HUMUS),2020-03-03 08:35:00,BOD ATU,BOD : 5 Day ATU,85,<,6.00,...,mg/l,FINAL SEWAGE EFFLUENT,True,WATER QUALITY OPERATOR SELF MONITORING COMPLIA...,403930.0,157510,2020,3,3,1.583224e+09
7764743,NaN,http://environment.data.gov.uk/water-quality/i...,SW-Z9420104,URCHFONT STW (HUMUS),2020-03-03 08:35:00,Ammonia(N),Ammoniacal Nitrogen as N,111,<,0.20,...,mg/l,FINAL SEWAGE EFFLUENT,True,WATER QUALITY OPERATOR SELF MONITORING COMPLIA...,403930.0,157510,2020,3,3,1.583224e+09


In [87]:
df_sw

,Site Number,Site Name,County,Folio Schedule,EA Number,Schedule Type,Year,Start Date,Start Time,End Date,End Time,Spill Count,Discharge Duration (hours),Discharge Period (hours),Catchment,StartDateTime,EndDateTime
0,103283,ABBEY ROAD FAVERSHAM CSO,KENT,A01377 / 1,A1377/K/05,Storm,2017,12/01/2017,14:35:58,12/01/2017,23:41:52,1,3.14,9.10,NaN,2017-01-12 14:35:58,2017-01-12 23:41:52
1,103283,ABBEY ROAD FAVERSHAM CSO,KENT,A01377 / 1,A1377/K/05,Storm,2017,17/05/2017,18:30:05,18/05/2017,22:31:54,2,4.33,28.03,NaN,2017-05-17 18:30:05,2017-05-18 22:31:54
2,103283,ABBEY ROAD FAVERSHAM CSO,KENT,A01377 / 1,A1377/K/05,Storm,2017,06/06/2017,09:11:33,06/06/2017,10:03:21,1,0.86,0.86,NaN,2017-06-06 09:11:33,2017-06-06 10:03:21
3,103283,ABBEY ROAD FAVERSHAM CSO,KENT,A01377 / 1,A1377/K/05,Storm,2017,27/06/2017,20:56:59,27/06/2017,22:07:46,1,1.18,1.18,NaN,2017-06-27 20:56:59,2017-06-27 22:07:46
4,103283,ABBEY ROAD FAVERSHAM CSO,KENT,A01377 / 1,A1377/K/05,Storm,2017,12/07/2017,05:22:19,12/07/2017,05:43:35,1,0.35,0.35,NaN,2017-07-12 05:22:19,2017-07-12 05:43:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27317,111906,YORK ROAD BEXHILL CSO [111906],EAST SUSSEX,A01265,A1265/S/05,Storm Sewage,Storm,24/09/2020,07:02:23,24/09/2020,09:05:00,1,2.04,2.04,NaN,2020-09-24 07:02:23,2020-09-24 09:05:00
27318,111906,YORK ROAD BEXHILL CSO [111906],EAST SUSSEX,A01265,A1265/S/05,Storm Sewage,Storm,25/10/2020,11:29:00,25/10/2020,12:22:00,1,0.88,0.88,NaN,2020-10-25 11:29:00,2020-10-25 12:22:00
27319,111906,YORK ROAD BEXHILL CSO [111906],EAST SUSSEX,A01265,A1265/S/05,Storm Sewage,Storm,15/11/2020,07:30:01,15/11/2020,11:42:22,1,4.21,4.21,NaN,2020-11-15 07:30:01,2020-11-15 11:42:22
27320,111906,YORK ROAD BEXHILL CSO [111906],EAST SUSSEX,A01265,A1265/S/05,Storm Sewage,Storm,04/12/2020,02:32:00,04/12/2020,06:57:00,1,4.42,4.42,NaN,2020-12-04 02:32:00,2020-12-04 06:57:00


In [246]:
# Filter down to langstone/portsmouth only

langdf_env = df_env[(df_env['sample.samplingPoint.label'].str.contains('LANGSTONE')) & (df_env['sampleDateTime'].dt.year >= 2017) & (df_env['sampleDateTime'].dt.year <= 2020)].copy()

langdf_sw = df_sw[(df_sw['Site Name'].str.contains('PORTSMOUTH')) & (df_sw['County'] == 'HAMPSHIRE')].copy()

# add 24 hours to sw end dates

langdf_sw['EndDateTime_plus1'] = langdf_sw['EndDateTime'] + datetime.timedelta(days=1)

# Sort data
langdf_env = langdf_env.sort_values('sampleDateTime')

In [252]:
def plotwithswbars(langdf_env, deter):
    fig, ax = plt.subplots()
    fig.set_size_inches(50., 10.5)
    langdf_env = langdf_env[langdf_env['determinand.label'] == deter]

    ax.plot(langdf_env['sampleDateTime'], langdf_env['result'], color='C0')

    # Plot bars
    for i, row in langdf_sw.iterrows():
        try:
            ax.axvspan(row['StartDateTime'], row['EndDateTime_plus1'], alpha=0.1, color='C7')
        except:
            pass
    
    # Vertical x ticks and limit to 15 in total
    plt.xticks(rotation='vertical')
    ax.xaxis.set_major_locator(plt.MaxNLocator(15))
    # plt.show()
    plt.savefig('environmentalplots/' + deter + '.png')
    plt.close()


# Plot every determinand in a loop
dets = langdf_env['determinand.label'].unique()
for i in range(len(dets)):
    plotwithswbars(langdf_env, dets[i])


In [ ]:
# # Join dataframes where the times are between the spillage times

# conn = sqlite3.connect(':memory:')
# #write the tables
# langdf_env.to_sql('env', conn, index=False)
# langdf_sw.to_sql('sw', conn, index=False)


# qry = '''
#     select  
#         *
#     from
#         env left join sw on
#         env.sampleDateTime between sw.StartDateTime and sw.EndDateTime_plus1 
#     '''
# df = pd.read_sql_query(qry, conn)

# # Object types will be lost, gotta convert back to datetimes

# df['StartDateTime'] = pd.to_datetime(df['StartDateTime'])
# df['EndDateTime'] = pd.to_datetime(df['EndDateTime'])
# df['EndDateTime_plus1'] = pd.to_datetime(df['EndDateTime_plus1'])
# df['sampleDateTime'] = pd.to_datetime(df['sampleDateTime'])

# # Sort
# df = df.sort_values('sampleDateTime')

# # Save
# df.to_csv('matcheddata.csv')

In [22]:
# Get data from southern water

# baseurl = 'https://www.southernwater.co.uk/gateway/Beachbuoy%20API/api/v1.0/Spills/GetHistoricSpills?page='
# npages=505

# resarr = []
# for i in range(0,npages):
#     r=requests.get(baseurl + str(i+1), headers={"x-gateway-apikey":"df32a632-3c29-4371-af71-3361b27d5baa"})
#     data = json.loads(r.content.decode('utf8'))
#     print(i)
#     resarr.extend(data['items'])
#     time.sleep(.1)

# with open('sw_data.json', 'w', encoding='utf-8') as f:
#     json.dump(resarr, f, ensure_ascii=False, indent=4)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27